In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**With Duplicate**

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Dropout, Concatenate, BatchNormalization, Lambda
)
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical

# ----------- DATA LOADING & DUPLICATION -----------
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)

def duplicate_dataset(data, num_duplicates):
    duplicated_data = data.tolist()
    for _ in range(num_duplicates):
        duplicated_data.extend(data.tolist())
    return np.array(duplicated_data)

num_duplicates = 1
data = duplicate_dataset(data, num_duplicates)

# ----------- FEATURES AND LABELS -----------
X_eeg   = np.array([entry['eeg_data']  for entry in data])
X_video = np.array([entry['video_data'] for entry in data]) # (N, 32, 64, 64)
X_ecg   = np.array([entry['ecg_data']  for entry in data])
y       = np.array([entry['labels']    for entry in data])
print(f"Number of samples after duplication: {len(data)}")

# Pad video data if needed
if X_video.shape[1:] != (32, 64, 64):
    padded_video = []
    for video in X_video:
        pad_shape = [
            (0, max(0, 32 - video.shape[0])),
            (0, max(0, 64 - video.shape[1])),
            (0, max(0, 64 - video.shape[2]))
        ]
        padded_video.append(np.pad(video, pad_shape, mode='constant'))
    X_video = np.array(padded_video)

# ----------- CLASS BALANCING -----------
def balance_classes(X1, X2, X3, y, class_idx):
    labels = y[:, class_idx]
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    max_count = np.max(class_counts)

    balanced_X1, balanced_X2, balanced_X3, balanced_y = [], [], [], []
    for cls in unique_classes:
        cls_mask = labels == cls
        cls_X1, cls_X2, cls_X3, cls_y = X1[cls_mask], X2[cls_mask], X3[cls_mask], y[cls_mask]

        if len(cls_y) < max_count:
            X1_oversampled, X2_oversampled, X3_oversampled, y_oversampled = resample(
                cls_X1, cls_X2, cls_X3, cls_y, replace=True,
                n_samples=max_count - len(cls_y), random_state=42)
            cls_X1 = np.concatenate([cls_X1, X1_oversampled])
            cls_X2 = np.concatenate([cls_X2, X2_oversampled])
            cls_X3 = np.concatenate([cls_X3, X3_oversampled])
            cls_y  = np.concatenate([cls_y,  y_oversampled])
        balanced_X1.append(cls_X1)
        balanced_X2.append(cls_X2)
        balanced_X3.append(cls_X3)
        balanced_y.append(cls_y)
    return (np.concatenate(balanced_X1),
            np.concatenate(balanced_X2),
            np.concatenate(balanced_X3),
            np.concatenate(balanced_y))

X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=1)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=2)

# ----------- SPLITTING -----------
X_eeg_train, X_eeg_test, X_video_train, X_video_test, X_ecg_train, X_ecg_test, y_train, y_test = \
    train_test_split(X_eeg, X_video, X_ecg, y, test_size=0.2, random_state=42)

num_valence_classes = int(np.max(y[:, 1])) + 1
num_arousal_classes = int(np.max(y[:, 2])) + 1

y_valence_train = to_categorical(y_train[:, 1], num_classes=num_valence_classes)
y_arousal_train = to_categorical(y_train[:, 2], num_classes=num_arousal_classes)
y_valence_test  = to_categorical(y_test[:, 1], num_classes=num_valence_classes)
y_arousal_test  = to_categorical(y_test[:, 2], num_classes=num_arousal_classes)

# ----------- VIDEO DATA FOR RESNET50V2 -----------
X_video_train_2d = X_video_train[:, 0, :, :][..., np.newaxis]  # [N, 64, 64, 1]
X_video_test_2d  = X_video_test[:, 0, :, :][..., np.newaxis]

# Vectorized batch preprocessing for ResNetV2
def resnet_preprocess(x):
    # x: array of shape (N, 64, 64, 1)
    x = np.repeat(x, 3, axis=-1)  # (N, 64, 64, 3)
    return preprocess_input(x)

X_video_train_2d = resnet_preprocess(X_video_train_2d)
X_video_test_2d  = resnet_preprocess(X_video_test_2d)

# ----------- MODEL DEFINITION -----------

# ==== Video branch: ResNet50V2 ====
video_input = Input(shape=(64, 64, 3), name="video_input")
resnet_base = ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(64, 64, 3),
    pooling='avg'
)
resnet_base.trainable = False  # Freeze for feature extraction
x = resnet_base(video_input)
cnn_output = Dense(128, activation='relu')(x)

# ==== EEG branch ====
eeg_input = Input(shape=(5, 512), name="eeg_input")
lstm_eeg = LSTM(128, return_sequences=True)(eeg_input)
lstm_eeg = Dropout(0.2)(lstm_eeg)
lstm_eeg = LSTM(64)(lstm_eeg)
lstm_eeg_output = Dense(128, activation='relu')(lstm_eeg)

# ==== ECG branch ====
ecg_input = Input(shape=(2, 512), name="ecg_input")
lstm_ecg = LSTM(64, return_sequences=True)(ecg_input)
lstm_ecg = Dropout(0.2)(lstm_ecg)
lstm_ecg = LSTM(32)(lstm_ecg)
lstm_ecg_output = Dense(64, activation='relu')(lstm_ecg)

# ==== Combine modalities ====
combined = Concatenate()([cnn_output, lstm_eeg_output, lstm_ecg_output])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.2)(combined)

# ==== Output layers ====
valence_output = Dense(num_valence_classes, activation='softmax', name="valence_output")(combined)
arousal_output = Dense(num_arousal_classes, activation='softmax', name="arousal_output")(combined)

model = Model(
    inputs=[video_input, eeg_input, ecg_input],
    outputs=[valence_output, arousal_output]
)

model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    metrics=['accuracy', 'accuracy']
)

model.summary()

# ----------- TRAINING -----------
history = model.fit(
    [X_video_train_2d, X_eeg_train, X_ecg_train],
    [y_valence_train, y_arousal_train],
    validation_data=(
        [X_video_test_2d, X_eeg_test, X_ecg_test],
        [y_valence_test, y_arousal_test]
    ),
    epochs=50,
    batch_size=128
)

# ----------- EVALUATION -----------
y_valence_pred, y_arousal_pred = model.predict([X_video_test_2d, X_eeg_test, X_ecg_test])
y_valence_pred_classes = np.argmax(y_valence_pred, axis=1)
y_arousal_pred_classes = np.argmax(y_arousal_pred, axis=1)
y_valence_true = np.argmax(y_valence_test, axis=1)
y_arousal_true = np.argmax(y_arousal_test, axis=1)

valence_accuracy = accuracy_score(y_valence_true, y_valence_pred_classes)
arousal_accuracy = accuracy_score(y_arousal_true, y_arousal_pred_classes)

print(f"Valence Accuracy: {valence_accuracy * 100:.2f}%")
print(f"Arousal Accuracy: {arousal_accuracy * 100:.2f}%")


Number of samples after duplication: 372
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ eeg_input           │ (None, 5, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 2, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 5, 128)    │    328,192 │ eeg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 2, 64)     │    147,712 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ video_input         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 5, 128)    │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 2, 64)     │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50v2          │ (None, 2048)      │ 23,564,800 │ video_input[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     49,408 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 32)        │     12,416 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    262,272 │ resnet50v2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      8,320 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      2,112 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 320)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │     82,176 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     32,896 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ valence_output      │ (None, 10)        │      1,290 │ dropout_3[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ arousal_output      │ (None, 10)        │      1,290 │ dropout_3[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,492,884 (93.43 MB)

 Trainable params: 928,084 (3.54 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - arousal_output_accuracy: 0.1203 - arousal_output_loss: 2.3164 - loss: 4.6944 - valence_output_accuracy: 0.1726 - valence_output_loss: 2.3666 - val_arousal_output_accuracy: 0.3452 - val_arousal_output_loss: 2.0370 - val_loss: 4.0378 - val_valence_output_accuracy: 0.3147 - val_valence_output_loss: 1.9930
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - arousal_output_accuracy: 0.2699 - arousal_output_loss: 2.0264 - loss: 3.9858 - valence_output_accuracy: 0.3377 - valence_output_loss: 1.9566 - val_arousal_output_accuracy: 0.4365 - val_arousal_output_loss: 1.8710 - val_loss: 3.6470 - val_valence_output_accuracy: 0.3350 - val_valence_output_loss: 1.7727
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - arousal_output_accuracy: 0.3934 - arousal_output_loss: 1.7646 - loss: 3.4893 - valence_output_accuracy: 0.4315 - valence_output_loss: 1.7136 - val_arousal_output_accuracy: 0.4772 - val_arousal_output_loss: 1.6604 - val_loss: 3.2102 - val_vale

**Without Duplicate**

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Dropout, Concatenate, BatchNormalization, Lambda
)
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical

# ----------- DATA LOADING -----------
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)

# ----------- FEATURES AND LABELS -----------
X_eeg   = np.array([entry['eeg_data']  for entry in data])
X_video = np.array([entry['video_data'] for entry in data]) # (N, 32, 64, 64)
X_ecg   = np.array([entry['ecg_data']  for entry in data])
y       = np.array([entry['labels']    for entry in data])
print(f"Number of samples after duplication: {len(data)}")

# Pad video data if needed
if X_video.shape[1:] != (32, 64, 64):
    padded_video = []
    for video in X_video:
        pad_shape = [
            (0, max(0, 32 - video.shape[0])),
            (0, max(0, 64 - video.shape[1])),
            (0, max(0, 64 - video.shape[2]))
        ]
        padded_video.append(np.pad(video, pad_shape, mode='constant'))
    X_video = np.array(padded_video)

# ----------- CLASS BALANCING -----------
def balance_classes(X1, X2, X3, y, class_idx):
    labels = y[:, class_idx]
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    max_count = np.max(class_counts)

    balanced_X1, balanced_X2, balanced_X3, balanced_y = [], [], [], []
    for cls in unique_classes:
        cls_mask = labels == cls
        cls_X1, cls_X2, cls_X3, cls_y = X1[cls_mask], X2[cls_mask], X3[cls_mask], y[cls_mask]

        if len(cls_y) < max_count:
            X1_oversampled, X2_oversampled, X3_oversampled, y_oversampled = resample(
                cls_X1, cls_X2, cls_X3, cls_y, replace=True,
                n_samples=max_count - len(cls_y), random_state=42)
            cls_X1 = np.concatenate([cls_X1, X1_oversampled])
            cls_X2 = np.concatenate([cls_X2, X2_oversampled])
            cls_X3 = np.concatenate([cls_X3, X3_oversampled])
            cls_y  = np.concatenate([cls_y,  y_oversampled])
        balanced_X1.append(cls_X1)
        balanced_X2.append(cls_X2)
        balanced_X3.append(cls_X3)
        balanced_y.append(cls_y)
    return (np.concatenate(balanced_X1),
            np.concatenate(balanced_X2),
            np.concatenate(balanced_X3),
            np.concatenate(balanced_y))

X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=1)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=2)

# ----------- SPLITTING -----------
X_eeg_train, X_eeg_test, X_video_train, X_video_test, X_ecg_train, X_ecg_test, y_train, y_test = \
    train_test_split(X_eeg, X_video, X_ecg, y, test_size=0.2, random_state=42)

num_valence_classes = int(np.max(y[:, 1])) + 1
num_arousal_classes = int(np.max(y[:, 2])) + 1

y_valence_train = to_categorical(y_train[:, 1], num_classes=num_valence_classes)
y_arousal_train = to_categorical(y_train[:, 2], num_classes=num_arousal_classes)
y_valence_test  = to_categorical(y_test[:, 1], num_classes=num_valence_classes)
y_arousal_test  = to_categorical(y_test[:, 2], num_classes=num_arousal_classes)

# ----------- VIDEO DATA FOR RESNET50V2 -----------
X_video_train_2d = X_video_train[:, 0, :, :][..., np.newaxis]  # [N, 64, 64, 1]
X_video_test_2d  = X_video_test[:, 0, :, :][..., np.newaxis]

# Vectorized batch preprocessing for ResNetV2
def resnet_preprocess(x):
    # x: array of shape (N, 64, 64, 1)
    x = np.repeat(x, 3, axis=-1)  # (N, 64, 64, 3)
    return preprocess_input(x)

X_video_train_2d = resnet_preprocess(X_video_train_2d)
X_video_test_2d  = resnet_preprocess(X_video_test_2d)

# ----------- MODEL DEFINITION -----------

# ==== Video branch: ResNet50V2 ====
video_input = Input(shape=(64, 64, 3), name="video_input")
resnet_base = ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(64, 64, 3),
    pooling='avg'
)
resnet_base.trainable = False  # Freeze for feature extraction
x = resnet_base(video_input)
cnn_output = Dense(128, activation='relu')(x)

# ==== EEG branch ====
eeg_input = Input(shape=(5, 512), name="eeg_input")
lstm_eeg = LSTM(128, return_sequences=True)(eeg_input)
lstm_eeg = Dropout(0.2)(lstm_eeg)
lstm_eeg = LSTM(64)(lstm_eeg)
lstm_eeg_output = Dense(128, activation='relu')(lstm_eeg)

# ==== ECG branch ====
ecg_input = Input(shape=(2, 512), name="ecg_input")
lstm_ecg = LSTM(64, return_sequences=True)(ecg_input)
lstm_ecg = Dropout(0.2)(lstm_ecg)
lstm_ecg = LSTM(32)(lstm_ecg)
lstm_ecg_output = Dense(64, activation='relu')(lstm_ecg)

# ==== Combine modalities ====
combined = Concatenate()([cnn_output, lstm_eeg_output, lstm_ecg_output])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.2)(combined)

# ==== Output layers ====
valence_output = Dense(num_valence_classes, activation='softmax', name="valence_output")(combined)
arousal_output = Dense(num_arousal_classes, activation='softmax', name="arousal_output")(combined)

model = Model(
    inputs=[video_input, eeg_input, ecg_input],
    outputs=[valence_output, arousal_output]
)

model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    metrics=['accuracy', 'accuracy']
)

model.summary()

# ----------- TRAINING -----------
history = model.fit(
    [X_video_train_2d, X_eeg_train, X_ecg_train],
    [y_valence_train, y_arousal_train],
    validation_data=(
        [X_video_test_2d, X_eeg_test, X_ecg_test],
        [y_valence_test, y_arousal_test]
    ),
    epochs=50,
    batch_size=128
)

# ----------- EVALUATION -----------
y_valence_pred, y_arousal_pred = model.predict([X_video_test_2d, X_eeg_test, X_ecg_test])
y_valence_pred_classes = np.argmax(y_valence_pred, axis=1)
y_arousal_pred_classes = np.argmax(y_arousal_pred, axis=1)
y_valence_true = np.argmax(y_valence_test, axis=1)
y_arousal_true = np.argmax(y_arousal_test, axis=1)

valence_accuracy = accuracy_score(y_valence_true, y_valence_pred_classes)
arousal_accuracy = accuracy_score(y_arousal_true, y_arousal_pred_classes)

print(f"Valence Accuracy: {valence_accuracy * 100:.2f}%")
print(f"Arousal Accuracy: {arousal_accuracy * 100:.2f}%")


Number of samples after duplication: 186


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ eeg_input           │ (None, 5, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 2, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 5, 128)    │    328,192 │ eeg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ (None, 2, 64)     │    147,712 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ video_input         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 5, 128)    │          0 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 2, 64)     │          0 │ lstm_6[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50v2          │ (None, 2048)      │ 23,564,800 │ video_input[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 64)        │     49,408 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 32)        │     12,416 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │    262,272 │ resnet50v2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │      8,320 │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      2,112 │ lstm_7[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 320)       │          0 │ dense_5[0][0],    │
│ (Concatenate)       │                   │            │ dense_6[0][0],    │
│                     │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 256)       │     82,176 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 256)       │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │     32,896 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 128)       │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ valence_output      │ (None, 10)        │      1,290 │ dropout_7[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ arousal_output      │ (None, 10)        │      1,290 │ dropout_7[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,492,884 (93.43 MB)

 Trainable params: 928,084 (3.54 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - arousal_output_accuracy: 0.1199 - arousal_output_loss: 2.3722 - loss: 4.8498 - valence_output_accuracy: 0.0939 - valence_output_loss: 2.4733 - val_arousal_output_accuracy: 0.2553 - val_arousal_output_loss: 2.1960 - val_loss: 4.3383 - val_valence_output_accuracy: 0.2234 - val_valence_output_loss: 2.1423
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - arousal_output_accuracy: 0.2209 - arousal_output_loss: 2.1492 - loss: 4.2943 - valence_output_accuracy: 0.2449 - valence_output_loss: 2.1441 - val_arousal_output_accuracy: 0.4043 - val_arousal_output_loss: 2.0134 - val_loss: 4.0152 - val_valence_output_accuracy: 0.3617 - val_valence_output_loss: 2.0018
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - arousal_output_accuracy: 0.3634 - arousal_output_loss: 1.9660 - loss: 3.9231 - valence_output_accuracy: 0.3513 - valence_output_loss: 1.9581 - val_arousal_output_accuracy: 0.4787 - val_arousal_output_loss: 1.8363 - val_loss: 3.7559 - val_valenc